In [42]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import keras.layers as L
from keras import optimizers, losses, metrics, Model
from keras.callbacks import EarlyStopping
from keras.models import Sequential, load_model
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [43]:
inputPath = "D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data"

commands = np.array(tf.io.gfile.listdir(str(inputPath)))
print(commands)
N_CLASSES = len(commands)
print(N_CLASSES)

['False' 'True']
2


In [44]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

filesArray = []
for path, subdirs, files in os.walk(inputPath):
    for name in files:
        filesArray.append(os.path.join(path + "\\", name))
print(len(filesArray))



for i in filesArray:
    print(i)

1004
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230513_161824.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230513_161901.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230513_161915.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230513_161925.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230513_161930.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230513_161934.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230513_161939.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230513_162008.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230513_162040.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\D

In [45]:
max_length = 1000
df = []
labels = []
for filename in filesArray:
    data = np.loadtxt(filename, dtype=np.float32)
    if len(data) < max_length:
        # pad the time series with zeros to a length of 1000
        data = np.pad(data, [(0, max_length - len(data)), (0, 0)], mode='constant')
    elif len(data) > max_length:
        # truncate the time series to a length of 1000
        data = data[:max_length, :]
    a = filename.split("\\")[-2]
    if a == "True":
        label = [1, 0]
    else:
        label = [0, 1]
    labels.append(label)
    df.append(data)

df_array = np.array(df)
labels_array = np.array(labels)

df_array, labels_array = unison_shuffled_copies(df_array, labels_array)

df_array, labels_array = unison_shuffled_copies(df_array, labels_array)


In [89]:
model = Sequential()
model.add(L.Input((1000, 1), name='input_accel'))
'''
# Rebuild Top
model.add(L.Conv1D(32,3, input_shape = (1000,1)))
model.add(L.Conv1D(8,8, input_shape = (1000,1)))
model.add(L.Dense(128, activation="relu"))
model.add(L.BatchNormalization())
model.add(L.Dense(32, activation="relu"))
model.add(L.Dropout(0.25))
model.add(L.GlobalMaxPooling1D())
model.add(L.Dense(2, activation="softmax"))
'''
model.add(L.Conv1D(16,2, activation="relu", input_shape = (1000,1)))
model.add(L.Dropout(0.2))
model.add(L.Conv1D(32,2, activation="relu", input_shape = (1000,1)))
model.add(L.GlobalMaxPooling1D())
model.add(L.Dropout(0.2))
model.add(L.Flatten())
model.add(L.Dense(32, activation="relu"))
#model.add(L.Dropout(0.2))
model.add(L.Dense(2, activation="softmax"))

model.compile(optimizer=tf.optimizers.Adam(learning_rate= 0.0001),
              loss=losses.CategoricalCrossentropy(),
              metrics=[metrics.CategoricalAccuracy()])
# model.build((235, 4001, 1))
print(model.summary())


df_array = np.array(df)
labels_array = np.array(labels)

model.fit(df_array, labels_array, shuffle=1, epochs=10, validation_split=0.15, batch_size = 1)

model.compile(optimizer=tf.optimizers.Adam(learning_rate= 0.000075),
              loss=losses.CategoricalCrossentropy(),
              metrics=[metrics.CategoricalAccuracy()])

model.fit(df_array, labels_array, shuffle=1, epochs=10, validation_split=0.15, batch_size = 2)

model.compile(optimizer=tf.optimizers.Adam(learning_rate= 0.00004),
              loss=losses.CategoricalCrossentropy(),
              metrics=[metrics.CategoricalAccuracy()])

model.fit(df_array, labels_array, shuffle=1, epochs=10, validation_split=0.15, batch_size = 4)



Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_80 (Conv1D)          (None, 999, 16)           48        
                                                                 
 dropout_101 (Dropout)       (None, 999, 16)           0         
                                                                 
 conv1d_81 (Conv1D)          (None, 998, 32)           1056      
                                                                 
 global_max_pooling1d_36 (Gl  (None, 32)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_102 (Dropout)       (None, 32)                0         
                                                                 
 flatten_34 (Flatten)        (None, 32)                0         
                                                     

In [91]:
model.evaluate(df_array, labels_array, batch_size=1)

1004/1004 [==============================] - 1s 693us/step - loss: 0.4553 - categorical_accuracy: 0.8775


[0.45528894662857056, 0.8774900436401367]